In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
gene_name="SOST"
URL= "https://www.genecards.org/cgi-bin/carddisp.pl?gene="+gene_name+"&keywords="+gene_name
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib 
print(soup.prettify()) 

<html style="height:100%">
 <head>
  <meta content="NOINDEX, NOFOLLOW" name="ROBOTS"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="initial-scale=1.0" name="viewport"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3" type="text/javascript">
  </script>
 </head>
 <body style="margin:0px;height:100%">
  <iframe frameborder="0" height="100%" id="main-iframe" marginheight="0px" marginwidth="0px" src="/_Incapsula_Resource?CWUDNSAI=22&amp;xinfo=12-14655-0%200NNN%20RT%281610994522306%2026%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B15%284%2c200%2c0%29%20U18&amp;incident_id=715000250000038027-86582292579916&amp;edet=15&amp;cinfo=04000000&amp;rpinfo=0" width="100%">
   Request unsuccessful. Incapsula incident ID: 715000250000038027-86582292579916
  </iframe>
 </body>
</html>
